In [4]:
# %pip uninstall scikit-learn -y



In [9]:
%pip install numpy==1.24.3 pandas==2.1.4 matplotlib==3.8.2 scikit-learn==1.3.2

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.29 requires numpy>=1.26.2; python_version < "3.13", but you have numpy 1.24.3 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached numpy-1.24.3-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp311-cp311-win_amd64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3


In [14]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import IsolationForest

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [11]:
data = pd.read_excel(r"C:\Users\Hp\anomaly_detection\Caseauditdetails_LIS 3.xlsx")

NameError: name 'pd' is not defined

In [ ]:
%pip install openpyxl

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
# Split implementation across modular cells below


In [ ]:
%pip install geopy

In [ ]:
%pip install faker

In [1]:
%pip uninstall -y numpy pandas scipy scikit-learn
%pip install numpy==1.26.4 pandas==2.1.4 scipy==1.11.4 scikit-learn==1.3.2

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.1.4
Uninstalling pandas-2.1.4:
  Successfully uninstalled pandas-2.1.4
Found existing installation: scipy 1.15.3
Uninstalling scipy-1.15.3:
  Successfully uninstalled scipy-1.15.3
Found existing installation: scikit-learn 1.3.2
Uninstalling scikit-learn-1.3.2:
  Successfully uninstalled scikit-learn-1.3.2
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/61.0 kB ? eta -:--:--
     --------------------------------- ------ 51.2/61.0 kB 2.7 MB/s eta 0:00:01
     --------------------------------- ------ 51.2/61.0 kB 2.7 MB/s eta 0:00:01
     -------------------------------------- 61.0/61.0 kB 406.4 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.4 kB ? eta -:--:--
     ------------------


[notice] A new release of pip is available: 23.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Imports
import pandas as pd
import numpy as np
from datetime import timedelta
from geopy.distance import geodesic
from sklearn.ensemble import IsolationForest
from faker import Faker
import random


In [ ]:
# Constants and configuration
fake = Faker()
roles = ['Pathologist Assistant', 'Admin', 'Lab Technician']
actions_allowed = {
    'Pathologist Assistant': ['Logged In', 'Viewed Final Report', 'Viewed Case Information', 'Revised as Addendum',
                              'Added Supplemental', 'Modified Diagnosis', 'Previewed Report'],
    'Admin': ['Logged In', 'Viewed Final Report', 'Modified Diagnosis', 'Added Supplemental', 'Create User', 'Delete User'],
    'Lab Technician': ['Logged In', 'Viewed Case Information', 'Collected Sample']
}
sensitive_actions = {'Revised as Addendum', 'Modified Diagnosis', 'Create User', 'Delete User'}
cities_coords = {
    'Hudson': (42.3770, -71.5661),
    'Boston': (42.3601, -71.0589),
    'New York': (40.7128, -74.0060),
    'Chicago': (41.8781, -87.6298),
    'San Francisco': (37.7749, -122.4194)
}
cities = list(cities_coords.keys())
user_regions = ['US', 'EU', 'ASIA']
organizations = ['Leominster Dermatology, LLP', 'Quantum Pathology Inc', 'Health Lab Solutions']
DATA_PATH = r"C:\Users\Hp\anomaly_detection\Caseauditdetails_LIS 3.xlsx"


In [ ]:
# Fraud detection logic
def determine_fraud_status(role, action, time_taken, audit_date, user_region):
    """
    Determine if a record should be labeled as fraud based on various patterns
    Returns 1 for fraud, 0 for not fraud
    """
    fraud_indicators = []
    
    # Pattern 1: Role violation (action not allowed for role)
    if role in actions_allowed:
        allowed_actions = actions_allowed[role]
        if action not in allowed_actions:
            fraud_indicators.append("role_violation")
    
    # Pattern 2: Suspicious timing (too fast execution)
    base_time = {'Pathologist Assistant': 4000, 'Admin': 6000, 'Lab Technician': 5000}
    expected_min = base_time.get(role, 5000) // 10
    if time_taken < expected_min:
        fraud_indicators.append("too_fast")
    
    # Pattern 3: After hours activity with sensitive actions
    sensitive_actions = {'Revised as Addendum', 'Modified Diagnosis', 'Create User', 'Delete User'}
    hour = audit_date.hour
    if (hour < 6 or hour > 22) and action in sensitive_actions:
        fraud_indicators.append("after_hours_sensitive")
    
    # Pattern 4: Multiple region activity (simplified - random chance for now)
    if user_region != 'US' and action in sensitive_actions:
        fraud_indicators.append("foreign_sensitive")
    
    # Pattern 5: Admin actions outside normal pattern
    if role != 'Admin' and action in ['Create User', 'Delete User']:
        fraud_indicators.append("unauthorized_admin")
    
    # Determine fraud based on indicators
    # 15% base fraud rate with multiple patterns possible
    if len(fraud_indicators) >= 2:
        return 1  # Definite fraud
    elif len(fraud_indicators) == 1:
        return 1 if random.random() < 0.7 else 0  # 70% chance of fraud
    else:
        return 1 if random.random() < 0.05 else 0  # 5% false positive rate


In [ ]:
# Synthetic helpers
def generate_action(role):
    if random.random() < 0.1:  # 10% anomaly role violation
        all_actions = set(sum(actions_allowed.values(), []))
        disallowed = list(all_actions - set(actions_allowed[role]))
        return random.choice(disallowed) if disallowed else random.choice(actions_allowed[role])
    else:
        return random.choice(actions_allowed[role])

def generate_time_taken(role):
    base = 5000
    if role == 'Pathologist Assistant': base = 4000
    elif role == 'Admin': base = 6000
    if random.random() < 0.05:  # 5% too-fast anomaly
        return random.randint(1, base // 10)
    return random.randint(base - 1000, base + 2000)

def synthetic_record(user_id, start_time):
    role = random.choice(roles)
    city = random.choice(cities)
    action = generate_action(role)
    time_taken = generate_time_taken(role)
    user_region = random.choice(user_regions)
    organization = random.choice(organizations)
    audit_date = start_time + timedelta(minutes=random.randint(0, 43200))  # last 30 days
    ip = Faker().ipv4()
    
    # Determine fraud status based on patterns
    is_fraud = determine_fraud_status(role, action, time_taken, audit_date, user_region)
    
    return {
        'userid': user_id,
        'formatteddisplayname': Faker().name(),
        'email': Faker().email(),
        'npi': 'UNKNOWN',
        'user region': user_region,
        'associationtype': 'Ordering Facility',
        'rolename': role,
        'organizationid': random.randint(15000, 16000),
        'caseid': random.randint(8200000, 8300000),
        'auditid': random.randint(180000000, 190000000),
        'actionperformed': action,
        'actiondetails': f"LoginId={Faker().user_name()};URL=https://lis.vitalaxis.com;IPAddress={ip};OS Version=Windows 10;Browser Version=Chrome 139;",
        'timetaken': time_taken,
        'auditdate': audit_date,
        'casestatus': 'Finalized' if random.random() > 0.4 else 'Addendum - Pending Sign-Out',
        'audittype': 'Cases',
        'accountid': random.randint(50000, 50050),
        'displayname': organization,
        'facility state': 'MA',
        'status': 'active',
        'orgid': random.randint(15000, 16000),
        'organizationname': organization,
        'city': city,
        'is_fraud': is_fraud  # Add fraud label
    }

def generate_synthetic_dataset(record_count=2000):
    start_time = pd.Timestamp.now() - timedelta(days=30)
    data = []
    user_count = record_count // 20
    for user_id in range(1000, 1000 + user_count):
        for _ in range(20):
            data.append(synthetic_record(user_id, start_time))
    return pd.DataFrame(data)


In [ ]:
# Load real or synthetic data
try:
    df = pd.read_excel(DATA_PATH)
except Exception as e:
    print("Using synthetic dataset (", e, ")")
    df = generate_synthetic_dataset(2000)

df.head(3)


In [ ]:
# Test the fraud-labeled dataset generation
print("Generating synthetic dataset with fraud labels...")
test_df = generate_synthetic_dataset(1000)  # Generate 1000 records for testing

print(f"\nDataset shape: {test_df.shape}")
print(f"\nColumns: {list(test_df.columns)}")

# Check fraud distribution
fraud_counts = test_df['is_fraud'].value_counts()
fraud_percentage = (fraud_counts[1] / len(test_df)) * 100 if 1 in fraud_counts else 0

print(f"\nFraud Distribution:")
print(f"Not Fraud (0): {fraud_counts.get(0, 0)} records")
print(f"Fraud (1): {fraud_counts.get(1, 0)} records")
print(f"Fraud Rate: {fraud_percentage:.2f}%")

# Show sample records
print(f"\nSample records:")
print(test_df[['userid', 'rolename', 'actionperformed', 'timetaken', 'auditdate', 'user region', 'is_fraud']].head(10))


In [ ]:
df.columns

In [ ]:
# If your DataFrame is named df:
df = df.drop(columns=['#', 'timetaken', 'displayname', 'orgid'], errors='ignore')
# or, in-place:
# df.drop(columns=['#', 'timetaken', 'displayname', 'orgid'], errors='ignore', inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
# Normalize fields
df['auditdate'] = pd.to_datetime(df['auditdate'])
# df['timeTakenMs'] = df['timetaken'].fillna(0).astype(int)
df['actionperformed'] = df['actionperformed'].fillna('Unknown')
df['rolename'] = df['rolename'].fillna('Unknown')
df['userId'] = df['userid'].astype(int)
df['ip'] = df['actiondetails'].str.extract(r'IPAddress=([^;]+)')[0].fillna('Unknown')
df['city'] = df['city'].fillna('Unknown')

# City lat/lng
df['lat'] = df['city'].map(lambda c: cities_coords.get(c, (np.nan, np.nan))[0])
df['lng'] = df['city'].map(lambda c: cities_coords.get(c, (np.nan, np.nan))[1])

df.head(3)


In [ ]:
pct_unknown = (df['ip'].isna() | (df['ip'] == 'Unknown')).mean() * 100
print(f"{pct_unknown:.2f}%")

In [ ]:
# Drop the 'ip' column
df.drop(columns=['ip'], errors='ignore', inplace=True)

In [ ]:
df.info()

In [ ]:
# Drop rows where caseid is 'Unknown' or NaN
import numpy as np
df = df.replace({'caseid': {'Unknown': np.nan}}).dropna(subset=['caseid'])

In [ ]:
df['caseid'] = df['caseid'].astype('int64')

In [ ]:
# Rolling features and flags prep
# def is_after_hours(ts):
#     return ts.hour < 9 or ts.hour >= 17

# # Sort for rolling stats
# df = df.sort_values(['userId', 'auditdate'])
# df['auditdate'] = pd.to_datetime(df['auditdate'], errors='coerce')


# R3 - actions in last 15 mins
# df['actions_15m'] = df.groupby('userId')['auditdate'].transform(lambda x: x.rolling('15min').count())

# 30-day rolling 95th percentile for R3
# df['p95_actions_30d'] = df.groupby('userId')['actions_15m'].transform(lambda x: x.rolling('30d').quantile(0.95)).fillna(0)

# # 30-day rolling minimum time for R4
# df['min_time_ms_30d'] = df.groupby('userId')['timeTakenMs'].transform(lambda x: x.rolling('30d').min()).fillna(0)

# df['after_hours'] = df['auditdate'].apply(is_after_hours)

# # Location key from lat/lng (rounded to reduce jitter)
# df['loc_key'] = (df['lat'].round(3).astype(str) + ',' + df['lng'].round(3).astype(str))

# # First time this location for the user?
# df['is_new_loc'] = ~df.groupby('userId')['loc_key'].apply(lambda s: s.duplicated(keep='first')).fillna(False)

# df[['is_new_loc','after_hours']].head()


In [ ]:
# # Rules R1–R6
# # Define the required variables first
# actions_allowed = {
#     'Pathologist Assistant': ['Logged In', 'Viewed Final Report', 'Viewed Case Information', 'Revised as Addendum',
#                               'Added Supplemental', 'Modified Diagnosis', 'Previewed Report'],
#     'Admin': ['Logged In', 'Viewed Final Report', 'Modified Diagnosis', 'Added Supplemental', 'Create User', 'Delete User'],
#     'Lab Technician': ['Logged In', 'Viewed Case Information', 'Collected Sample']
# }

# sensitive_actions = {'Revised as Addendum', 'Modified Diagnosis', 'Create User', 'Delete User'}

# allowed_actions = {k:set(v) for k, v in actions_allowed.items()}

# # R1: Role violation
# df['R1_flag'] = ~df.apply(lambda r: r['actionperformed'] in allowed_actions.get(r['rolename'], set()), axis=1)

# # R2: New country + sensitive action - simplified as just sensitive action here
# def r2_flag(group):
#     seen_countries = set()
#     flags = []
#     for _, row in group.iterrows():
#         new_country = row['user region'] not in seen_countries
#         flag = new_country and (row['actionperformed'] in sensitive_actions)
#         flags.append(flag)
#         seen_countries.add(row['user region'])
#     return pd.Series(flags, index=group.index)

# # Fix the groupby issue by ensuring we get a Series
# df['R2_flag'] = df.groupby('userId', group_keys=False).apply(r2_flag)

# # # R3: Volume spike
# # df['R3_flag'] = df['actions_15m'] > (df['p95_actions_30d'] + 3)

# # # R4: Too fast execution (margin = 100ms)
# # margin_ms = 100
# # df['R4_flag'] = df['timeTakenMs'] < (df['min_time_ms_30d'] - margin_ms)

# # # R5: After hours + new IP
# # df['R5_flag'] = df['after_hours'] & df['is_new_ip']

# # R6: Geo-velocity > 1500km in 30 mins
# def geo_velocity_flag(user_df):
#     flags = []
#     for idx, row in user_df.iterrows():
#         flagged = False
#         lat1, lng1 = row['lat'], row['lng']
#         if np.isnan(lat1) or np.isnan(lng1):
#             flags.append(False)
#             continue
#         recent = user_df[(user_df['auditdate'] < row['auditdate']) &
#                          (user_df['auditdate'] >= row['auditdate'] - timedelta(minutes=30))]
#         for _, prev_row in recent.iterrows():
#             lat2, lng2 = prev_row['lat'], prev_row['lng']
#             if np.isnan(lat2) or np.isnan(lng2):
#                 continue
#             dist = geodesic((lat1, lng1), (lat2, lng2)).km
#             if dist > 1500:
#                 flagged = True
#                 break
#         flags.append(flagged)
#     return pd.Series(flags, index=user_df.index)

# df['R6_flag'] = df.groupby('userId', group_keys=False).apply(geo_velocity_flag)

# df[[c for c in df.columns if c.endswith('_flag')]].sum()


In [ ]:
# Isolation Forest - Feature Analysis for Anomaly Detection

import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

print("DataFrame shape:", df.shape)
print("\nAvailable columns:")
print(df.columns.tolist())

# Check data types
print("\nData types:")
print(df.dtypes)

# Analyze potential features for Isolation Forest
print("\n" + "="*50)
print("ANOMALY DETECTION FEATURES ANALYSIS")
print("="*50)

# 1. TEMPORAL FEATURES (Time-based anomalies)
print("\n1. TEMPORAL FEATURES:")
print("- Hour of day (df['auditdate'].dt.hour)")
print("- Day of week (df['auditdate'].dt.dayofweek)")
print("- Time between actions (if multiple actions per user)")

# 2. BEHAVIORAL FEATURES (User behavior patterns)
print("\n2. BEHAVIORAL FEATURES:")
print("- Action frequency per user")
print("- Number of different actions performed")
print("- Session duration patterns")
print("- Geographic patterns (city changes)")

# 3. ROLE-BASED FEATURES (Role-specific anomalies)
print("\n3. ROLE-BASED FEATURES:")
print("- Action count per role")
print("- Unusual action sequences for role")
print("- Time spent on actions (if available)")

# 4. GEOGRAPHIC FEATURES (Location anomalies)
print("\n4. GEOGRAPHIC FEATURES:")
print("- Latitude/Longitude coordinates")
print("- Distance between consecutive actions")
print("- New location detection")

# 5. ORGANIZATIONAL FEATURES (Organization patterns)
print("\n5. ORGANIZATIONAL FEATURES:")
print("- Organization size (number of users)")
print("- Action distribution across organizations")
print("- User region patterns")

print("\n" + "="*50)
print("RECOMMENDED FEATURES FOR ISOLATION FOREST:")
print("="*50)


In [ ]:
# Feature Engineering for Isolation Forest

# 1. TEMPORAL FEATURES
df['hour_of_day'] = df['auditdate'].dt.hour
df['day_of_week'] = df['auditdate'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_after_hours'] = ((df['hour_of_day'] < 9) | (df['hour_of_day'] >= 17)).astype(int)

# 2. USER BEHAVIOR FEATURES
# Count actions per user
user_action_counts = df.groupby('userId').size().to_dict()
df['user_action_frequency'] = df['userId'].map(user_action_counts)

# Count unique actions per user
user_unique_actions = df.groupby('userId')['actionperformed'].nunique().to_dict()
df['user_unique_actions'] = df['userId'].map(user_unique_actions)

# Count unique cities per user (geographic diversity)
user_cities = df.groupby('userId')['city'].nunique().to_dict()
df['user_geographic_diversity'] = df['userId'].map(user_cities)

# 3. ROLE ENCODING
le_role = LabelEncoder()
df['role_encoded'] = le_role.fit_transform(df['rolename'])

# 4. ACTION ENCODING
le_action = LabelEncoder()
df['action_encoded'] = le_action.fit_transform(df['actionperformed'])

# 5. ORGANIZATION FEATURES
org_user_counts = df.groupby('organizationid').size().to_dict()
df['org_size'] = df['organizationid'].map(org_user_counts)

# 6. GEOGRAPHIC FEATURES (using lat/lng if available)
if 'lat' in df.columns and 'lng' in df.columns:
    df['has_coordinates'] = (~df['lat'].isna() & ~df['lng'].isna()).astype(int)
else:
    df['has_coordinates'] = 0

print("Features created:")
print("- hour_of_day, day_of_week, is_weekend, is_after_hours")
print("- user_action_frequency, user_unique_actions, user_geographic_diversity")
print("- role_encoded, action_encoded")
print("- org_size, has_coordinates")


In [ ]:
# Isolation Forest Implementation

# Select features for anomaly detection
isolation_features = [
    'hour_of_day', 'day_of_week', 'is_weekend', 'is_after_hours',
    'user_action_frequency', 'user_unique_actions', 'user_geographic_diversity',
    'role_encoded', 'action_encoded', 'org_size', 'has_coordinates'
]

# Check which features are available
available_features = [f for f in isolation_features if f in df.columns]
print(f"Using {len(available_features)} features for Isolation Forest:")
print(available_features)

# Prepare feature matrix
X = df[available_features].copy()

# Handle any missing values
X = X.fillna(0)

print(f"\nFeature matrix shape: {X.shape}")
print(f"Missing values: {X.isnull().sum().sum()}")

# Initialize and fit Isolation Forest
# contamination: proportion of anomalies expected (5% = 0.05)
iso_forest = IsolationForest(
    contamination=0.1,  # Expect 10% of data to be anomalies
    random_state=42,
    n_estimators=100
)

# Fit the model
iso_forest.fit(X)

# Predict anomalies (-1 = anomaly, 1 = normal)
anomaly_predictions = iso_forest.predict(X)
anomaly_scores = iso_forest.decision_function(X)

# Add results to dataframe
df['isolation_forest_anomaly'] = (anomaly_predictions == -1).astype(int)
df['isolation_forest_score'] = anomaly_scores

# Results summary
n_anomalies = df['isolation_forest_anomaly'].sum()
n_total = len(df)
anomaly_percentage = (n_anomalies / n_total) * 100

print(f"\n" + "="*50)
print("ISOLATION FOREST RESULTS")
print("="*50)
print(f"Total records: {n_total}")
print(f"Anomalies detected: {n_anomalies}")
print(f"Anomaly percentage: {anomaly_percentage:.2f}%")
print(f"Normal records: {n_total - n_anomalies}")

# Show feature importance (based on average depth)
feature_importances = np.abs(iso_forest.estimators_[0].feature_importances_)
feature_importance_df = pd.DataFrame({
    'feature': available_features,
    'importance': feature_importances
}).sort_values('importance', ascending=False)

print(f"\nFeature Importances:")
print(feature_importance_df)


In [ ]:
# Analyze and Visualize Detected Anomalies

print("\n" + "="*60)
print("DETAILED ANOMALY ANALYSIS")
print("="*60)

# 1. Anomalies by Role
print("\n1. ANOMALIES BY ROLE:")
role_anomalies = df.groupby('rolename')['isolation_forest_anomaly'].agg(['count', 'sum', 'mean']).round(3)
role_anomalies.columns = ['Total_Actions', 'Anomalies', 'Anomaly_Rate']
print(role_anomalies)

# 2. Anomalies by Action Type
print("\n2. ANOMALIES BY ACTION TYPE:")
action_anomalies = df.groupby('actionperformed')['isolation_forest_anomaly'].agg(['count', 'sum', 'mean']).round(3)
action_anomalies.columns = ['Total_Actions', 'Anomalies', 'Anomaly_Rate']
action_anomalies = action_anomalies[action_anomalies['Total_Actions'] >= 2].sort_values('Anomaly_Rate', ascending=False)
print(action_anomalies.head(10))

# 3. Anomalies by Time of Day
print("\n3. ANOMALIES BY HOUR OF DAY:")
hour_anomalies = df.groupby('hour_of_day')['isolation_forest_anomaly'].agg(['count', 'sum', 'mean']).round(3)
hour_anomalies.columns = ['Total_Actions', 'Anomalies', 'Anomaly_Rate']
print(hour_anomalies)

# 4. Top Anomalous Records
print("\n4. TOP 10 MOST ANOMALOUS RECORDS:")
anomalous_records = df[df['isolation_forest_anomaly'] == 1].nlargest(10, 'isolation_forest_score')
display_cols = ['userId', 'rolename', 'actionperformed', 'city', 'hour_of_day', 'isolation_forest_score']
print(anomalous_records[display_cols].to_string())

# 5. Anomaly Score Distribution
print("\n5. ANOMALY SCORE DISTRIBUTION:")
print(f"Minimum score: {df['isolation_forest_score'].min():.3f}")
print(f"Maximum score: {df['isolation_forest_score'].max():.3f}")
print(f"Mean score: {df['isolation_forest_score'].mean():.3f}")
print(f"Standard deviation: {df['isolation_forest_score'].std():.3f}")

# 6. Anomalies by Organization
print("\n6. ANOMALIES BY ORGANIZATION:")
if 'organizationname' in df.columns:
    org_anomalies = df.groupby('organizationname')['isolation_forest_anomaly'].agg(['count', 'sum', 'mean']).round(3)
    org_anomalies.columns = ['Total_Actions', 'Anomalies', 'Anomaly_Rate']
    org_anomalies = org_anomalies.sort_values('Anomaly_Rate', ascending=False)
    print(org_anomalies.head(5))

# 7. Geographic Anomalies
print("\n7. ANOMALIES BY CITY:")
city_anomalies = df.groupby('city')['isolation_forest_anomaly'].agg(['count', 'sum', 'mean']).round(3)
city_anomalies.columns = ['Total_Actions', 'Anomalies', 'Anomaly_Rate']
city_anomalies = city_anomalies[city_anomalies['Total_Actions'] >= 2].sort_values('Anomaly_Rate', ascending=False)
print(city_anomalies.head(5))

print("\n" + "="*60)
print("PURE ML ANOMALY DETECTION COMPLETE!")
print("="*60)


In [ ]:
# Visualization of Anomalies

import matplotlib.pyplot as plt
import seaborn as sns

# Set up the plotting style
plt.style.use('default')
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Isolation Forest Anomaly Detection Results', fontsize=16, fontweight='bold')

# 1. Anomaly Score Distribution
axes[0,0].hist(df['isolation_forest_score'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].axvline(df[df['isolation_forest_anomaly']==1]['isolation_forest_score'].min(), 
                  color='red', linestyle='--', linewidth=2, label='Anomaly Threshold')
axes[0,0].set_title('Distribution of Anomaly Scores')
axes[0,0].set_xlabel('Anomaly Score')
axes[0,0].set_ylabel('Frequency')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Anomalies by Hour of Day
hour_anomaly_data = df.groupby('hour_of_day')['isolation_forest_anomaly'].mean().reset_index()
axes[0,1].bar(hour_anomaly_data['hour_of_day'], hour_anomaly_data['isolation_forest_anomaly'], 
              color='lightcoral', alpha=0.7)
axes[0,1].set_title('Anomaly Rate by Hour of Day')
axes[0,1].set_xlabel('Hour of Day')
axes[0,1].set_ylabel('Anomaly Rate')
axes[0,1].set_xticks(range(0, 24, 2))
axes[0,1].grid(True, alpha=0.3)

# 3. Anomalies by Role
role_anomaly_data = df.groupby('rolename')['isolation_forest_anomaly'].mean().reset_index()
bars = axes[1,0].bar(role_anomaly_data['rolename'], role_anomaly_data['isolation_forest_anomaly'], 
                     color='lightgreen', alpha=0.7)
axes[1,0].set_title('Anomaly Rate by Role')
axes[1,0].set_xlabel('Role')
axes[1,0].set_ylabel('Anomaly Rate')
axes[1,0].tick_params(axis='x', rotation=45)
axes[1,0].grid(True, alpha=0.3)

# 4. Scatter plot: User Action Frequency vs Anomaly Score
scatter = axes[1,1].scatter(df['user_action_frequency'], df['isolation_forest_score'], 
                           c=df['isolation_forest_anomaly'], cmap='RdYlBu_r', alpha=0.6)
axes[1,1].set_title('User Action Frequency vs Anomaly Score')
axes[1,1].set_xlabel('User Action Frequency')
axes[1,1].set_ylabel('Anomaly Score')
axes[1,1].grid(True, alpha=0.3)
plt.colorbar(scatter, ax=axes[1,1], label='Anomaly (1=Yes, 0=No)')

plt.tight_layout()
plt.show()

# Summary statistics
print("\n" + "="*50)
print("ANOMALY DETECTION SUMMARY")
print("="*50)
print(f"Total Records Analyzed: {len(df)}")
print(f"Anomalies Detected: {df['isolation_forest_anomaly'].sum()}")
print(f"Anomaly Rate: {(df['isolation_forest_anomaly'].sum() / len(df) * 100):.2f}%")
print(f"Normal Records: {len(df) - df['isolation_forest_anomaly'].sum()}")

# Most anomalous users
print(f"\nTop 5 Most Anomalous Users:")
top_anomalous_users = df[df['isolation_forest_anomaly'] == 1].groupby('userId')['isolation_forest_score'].max().nlargest(5)
for user_id, score in top_anomalous_users.items():
    user_actions = df[df['userId'] == user_id]['actionperformed'].unique()
    print(f"User {user_id}: Score {score:.3f}, Actions: {list(user_actions)}")
